In [1]:
import numpy as np
import pyldpc as ldpc
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import sys
from utils_f import load_code
import os
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
device='cpu'

In [18]:
H_filename = './source/LDPC_N96_K48_P8_set0_dmin10.alist'
code = load_code(H_filename)
H = code.H
G = code.G
var_degrees = code.var_degrees
chk_degrees = code.chk_degrees
num_edges = code.num_edges
u = code.u
d = code.d
n = code.n
m = code.m
k = code.k

In [19]:
class min_sum(torch.nn.Module):
    def __init__(self, num_iterations = 5):
        super(min_sum, self).__init__()
        self.W_cv = torch.ones((num_iterations, num_edges))
        # self.W_cv = torch.nn.Parameter(torch.abs(torch.ones((num_iterations, num_edges))))
        # self.W_cv.requires_grad = True
        self.B_cv = torch.zeros((num_iterations, num_edges))
        # self.B_cv = torch.nn.Parameter(torch.abs(torch.zeros((num_iterations, num_edges))))
        # self.B_cv.requires_grad = True
        self.W_vc = torch.ones((num_iterations, num_edges))
        # self.W_vc = torch.nn.Parameter(torch.abs(torch.ones((num_iterations, num_edges))))
        # self.W_vc.requires_grad = True
        
        self.num_iterations = num_iterations
        
    def forward(self, soft_input):
        soft_input = soft_input.T           # TODO! Fix matrix dimensions inside functions
        # print(soft_input.shape)
        cv = torch.zeros((num_edges, soft_input.shape[1]))
        for iteration in range(0, self.num_iterations):
            vc = self.compute_vc(cv, soft_input, iteration)
            cv = self.compute_cv(vc, iteration)
            soft_input = self.marginalize(soft_input, cv)
        output = torch.sigmoid(-soft_input)
        output = output.T
        
        # self.W_cv = torch.nn.Parameter(torch.abs(self.W_cv))
        # self.B_cv =torch.nn.Parameter(torch.abs(self.B_cv))
        # self.W_vc = torch.nn.Parameter(torch.abs(self.W_vc))
        # output = soft_input
        return output

        
    def compute_vc(self, cv, soft_input, iteration):
        edges = []
        for i in range(0, n):
            for j in range(0, var_degrees[i]):
                edges.append(i)
        # print(soft_input.shape, len(edges), edges)
        reordered_soft_input = torch.index_select(soft_input, 0, torch.tensor(edges).to(device))

        vc = torch.zeros((num_edges, cv.shape[1])).to(device)
        counter = 0
        edge_order = []

        for i in range(0, n): # for each variable node v
            for j in range(0, var_degrees[i]):
                edge_order.append(d[i][j])
                extrinsic_edges = []
                for jj in range(0, var_degrees[i]):
                    if jj != j: # extrinsic information only
                        extrinsic_edges.append(d[i][jj])
                # if the list of edges is not empty, add them up
                if extrinsic_edges:
                    # print(cv.shape, len(extrinsic_edges), extrinsic_edges)
                    temp = torch.index_select(cv.to(device), 0, torch.tensor(extrinsic_edges).to(device))
                    temp = torch.sum(temp, 0)
                else:
                    temp = torch.zeros(cv.shape[1])

                temp = temp.to(device)
                vc[counter] = temp
        
        new_order = np.zeros(num_edges).astype(int)
        new_order[edge_order] = np.arange(0, num_edges)
        vc = torch.index_select(vc, 0, torch.tensor(new_order).to(device))
        vc += reordered_soft_input * torch.tile(torch.reshape(self.W_vc[iteration], (-1,1)), (1, cv.shape[1])).to(device)       # add soft inputs of the previous iterations!
        return vc 

    def compute_cv(self, vc, iteration):
        cv_list = []
        prod_list = []
        min_list = []
        edge_order = []
        for i in range(0, m): # for each check node c
            for j in range(0, chk_degrees[i]):
                edge_order.append(u[i][j])
                extrinsic_edges = []
                for jj in range(0, chk_degrees[i]):
                    if jj != j:
                        extrinsic_edges.append(u[i][jj])
                temp = torch.index_select(vc.to(device),0,torch.tensor(extrinsic_edges).to(device))
                temp1 = torch.prod(torch.sign(temp),0)
                temp2 = torch.min(torch.abs(temp),0)[0]
                prod_list.append(temp1)
                min_list.append(temp2)
        prods = torch.stack(prod_list)
        mins = torch.stack(min_list)
        mins = torch.relu(mins - torch.tile(torch.reshape(self.B_cv[iteration], (-1,1)), (1, vc.shape[1])).to(device))
        cv = prods * mins
        new_order = np.zeros(num_edges).astype(int)
        new_order[edge_order] = np.array(range(0,num_edges)).astype(int)
        cv = torch.index_select(cv, 0, torch.tensor(new_order).to(device))
        cv = cv * torch.tile(torch.reshape(self.W_cv[iteration], (-1,1)), (1, vc.shape[1])).to(device)
        return cv

    # combine messages to get posterior LLRs
    def marginalize(self, soft_input, cv):
        weighted_soft_input = soft_input
        soft_output = []
        for i in range(0,n):
            edges = []
            for e in range(0,var_degrees[i]):
                edges.append(d[i][e])
            temp = torch.index_select(cv,0,torch.tensor(edges).to(device))
            temp = torch.sum(temp, 0)
            soft_output.append(temp)
        soft_output = torch.stack(soft_output)
        soft_output = weighted_soft_input + soft_output
        return soft_output

In [20]:
class Decoder(torch.nn.Module):
    def __init__(self, num_iterations = 5):
        super(Decoder, self).__init__()
        # self.W_cv = torch.ones((num_iterations, num_edges))
        self.W_cv = torch.nn.Parameter(torch.abs(torch.randn((num_iterations, num_edges))))
        self.W_cv.requires_grad = True
        # self.B_cv = torch.zeros((num_iterations, num_edges))
        self.B_cv = torch.nn.Parameter(torch.abs(torch.randn((num_iterations, num_edges))))
        self.B_cv.requires_grad = True
        # self.W_vc = torch.ones((num_iterations, num_edges))
        self.W_vc = torch.nn.Parameter(torch.abs(torch.randn((num_iterations, num_edges))))
        self.W_vc.requires_grad = True
        
        self.num_iterations = num_iterations
        
    def forward(self, soft_input):
        soft_input = soft_input.T           # TODO! Fix matrix dimensions inside functions
        # print(soft_input.shape)
        cv = torch.zeros((num_edges, soft_input.shape[1]))
        for iteration in range(0, self.num_iterations):
            vc = self.compute_vc(cv, soft_input, iteration)
            cv = self.compute_cv(vc, iteration)
            soft_input = self.marginalize(soft_input, cv)
        output = torch.sigmoid(-soft_input)
        output = output.T
        
        # self.W_cv = torch.nn.Parameter(torch.abs(self.W_cv))
        # self.B_cv =torch.nn.Parameter(torch.abs(self.B_cv))
        # self.W_vc = torch.nn.Parameter(torch.abs(self.W_vc))
        
        # output = soft_input
        return output

        
    def compute_vc(self, cv, soft_input, iteration):
        edges = []
        for i in range(0, n):
            for j in range(0, var_degrees[i]):
                edges.append(i)
        # print(soft_input.shape, len(edges), edges)
        reordered_soft_input = torch.index_select(soft_input, 0, torch.tensor(edges).to(device))

        vc = torch.zeros((num_edges, cv.shape[1])).to(device)
        counter = 0
        edge_order = []

        for i in range(0, n): # for each variable node v
            for j in range(0, var_degrees[i]):
                edge_order.append(d[i][j])
                extrinsic_edges = []
                for jj in range(0, var_degrees[i]):
                    if jj != j: # extrinsic information only
                        extrinsic_edges.append(d[i][jj])
                # if the list of edges is not empty, add them up
                if extrinsic_edges:
                    # print(cv.shape, len(extrinsic_edges), extrinsic_edges)
                    temp = torch.index_select(cv.to(device), 0, torch.tensor(extrinsic_edges).to(device))
                    temp = torch.sum(temp, 0)
                else:
                    temp = torch.zeros(cv.shape[1])

                temp = temp.to(device)
                vc[counter] = temp
        
        new_order = np.zeros(num_edges).astype(int)
        new_order[edge_order] = np.arange(0, num_edges)
        vc = torch.index_select(vc, 0, torch.tensor(new_order).to(device))
        vc += reordered_soft_input * torch.tile(torch.reshape(self.W_vc[iteration], (-1,1)), (1, cv.shape[1]))       # add soft inputs of the previous iterations!
        return vc 

    def compute_cv(self, vc, iteration):
        cv_list = []
        prod_list = []
        min_list = []
        edge_order = []
        for i in range(0, m): # for each check node c
            for j in range(0, chk_degrees[i]):
                edge_order.append(u[i][j])
                extrinsic_edges = []
                for jj in range(0, chk_degrees[i]):
                    if jj != j:
                        extrinsic_edges.append(u[i][jj])
                temp = torch.index_select(vc.to(device),0,torch.tensor(extrinsic_edges).to(device))
                temp1 = torch.prod(torch.sign(temp),0)
                temp2 = torch.min(torch.abs(temp),0)[0]
                prod_list.append(temp1)
                min_list.append(temp2)
        prods = torch.stack(prod_list)
        mins = torch.stack(min_list)
        mins = torch.relu(mins - torch.tile(torch.reshape(self.B_cv[iteration], (-1,1)), (1, vc.shape[1])))
        cv = prods * mins
        new_order = np.zeros(num_edges).astype(int)
        new_order[edge_order] = np.array(range(0,num_edges)).astype(int)
        cv = torch.index_select(cv, 0, torch.tensor(new_order).to(device))
        cv = cv * torch.tile(torch.reshape(self.W_cv[iteration], (-1,1)), (1, vc.shape[1]))
        return cv

    # combine messages to get posterior LLRs
    def marginalize(self, soft_input, cv):
        weighted_soft_input = soft_input
        soft_output = []
        for i in range(0,n):
            edges = []
            for e in range(0,var_degrees[i]):
                edges.append(d[i][e])
            temp = torch.index_select(cv,0,torch.tensor(edges).to(device))
            temp = torch.sum(temp, 0)
            soft_output.append(temp)
        soft_output = torch.stack(soft_output)
        soft_output = weighted_soft_input + soft_output
        return soft_output

In [21]:
def epoch_train(loader, clf, criterion, opt):
    clf.train(True)
    avg_loss = 0
    avg_acc = 0
    correct = 0
    # load batch
    for model_input, target in loader:
        # move data to device
        model_input = model_input.to(device)
        target = target.to(device)
        # calculate outputs, loss and accuracy
        model_output = clf(model_input)
        loss = criterion(model_output, target)
        # print(model_input[0], model_output[0])
        avg_loss += loss
        correct += torch.count_nonzero(torch.heaviside(model_output-0.5, torch.tensor([0.]).to(device)).to(device) == target)
        # calculate grad, upd weights
        opt.zero_grad()
        loss.backward()
        opt.step()
    avg_loss = avg_loss / len(loader)
    avg_acc = correct / len(loader.dataset)
    return avg_loss, avg_acc

            
            
                        
def epoch_test(loader, clf, criterion):
    clf.eval()
    avg_loss = 0
    avg_acc = 0
    correct = 0
    # load batch
    for model_input, target in loader:
        # move data to device
        model_input = model_input.to(device)
        target = target.to(device)
        # calculate outputs, loss and accuracy
        model_output = clf(model_input)
        loss = criterion(model_output, target)
        avg_loss += loss
        correct += torch.count_nonzero(torch.heaviside(model_output-0.5, torch.tensor([0.]).to(device)).to(device) == target)
    avg_loss = avg_loss / len(loader)
    avg_acc = correct / len(loader.dataset)

    return avg_loss, avg_acc

def train(train_loader, test_loader, clf, criterion, opt, n_epochs=50):
    for epoch in trange(n_epochs):
        train_loss, train_acc = epoch_train(train_loader, clf, criterion, opt)
        test_loss, test_acc = epoch_test(test_loader, clf, criterion)
        # clf.apply(constraints)

        # clf.W_cv = torch.nn.Parameter(torch.relu(clf.W_cv))
        # clf.B_cv = torch.nn.Parameter(torch.relu(clf.B_cv))
        # clf.W_vc = torch.nn.Parameter(torch.relu(clf.W_vc))
        # print(clf.W_cv[0])


        if (np.mod(epoch+1,1)==0):
            print(f'[Epoch {epoch + 1}] train loss: {train_loss:.3f}; train acc: {train_acc:.2f}; ' + 
                  f'test loss: {test_loss:.3f}; test acc: {test_acc:.2f}')

## Create dataloader

In [22]:
def gen():
    # messages = np.random.randint(0,2,[train_size,k])
    # codewords = messages @ G % 2
    # BPSK_codewords = (0.5 - codewords.astype(np.float32)) * 2.0
    codewords = torch.ones((train_size, n)).to(device)
    BPSK_codewords = -1*torch.ones_like(codewords).to(device)
    soft_input = torch.zeros_like(BPSK_codewords).to(device)
    SNRs = np.arange(1,6)
    for i in range(0,len(SNRs)):
        sigma = torch.sqrt(torch.tensor(1) / (2 * (k/n) * 10**(SNRs[i]/10))).to(device)
        noise = sigma * torch.randn((train_size//len(SNRs),n)).to(device)
        start_idx = train_size*i//len(SNRs)
        end_idx = train_size*(i+1)//len(SNRs)
        soft_input[start_idx:end_idx,:] = BPSK_codewords[start_idx:end_idx,:] + noise
        soft_input = 2 * soft_input / (sigma**2)
    return soft_input, codewords
    
def update_loaders():

    soft_input_train, codewords_train = gen()
    train_X = soft_input_train # transform to torch tensor
    train_y = codewords_train
    training_dataset = torch.utils.data.TensorDataset(train_X, train_y) # create your datset
    training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=20000, shuffle=True) # create your dataloader


    soft_input_test, codewords_test = gen()
    test_X = soft_input_test # transform to torch tensor
    test_y = codewords_test
    test_dataset = torch.utils.data.TensorDataset(test_X, test_y) # create your datset
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=20000, shuffle=True) # create your dataloader
    return training_loader, test_loader

In [23]:
decoder = Decoder(num_iterations=5).to(device)
# opt = torch.optim.Adam(decoder.parameters(), lr=0.0025)
opt = torch.optim.SGD(decoder.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()

In [24]:
test_size = 10000
req_err = 500*n
SNRs = np.arange(-10,5)

In [25]:
# hard decisions, classic min-sum
ms = min_sum(num_iterations=20).to(device)

frame_errors_hard = torch.zeros(len(SNRs)).to(device)
nb_frames_hard = torch.zeros(len(SNRs)).to(device)
frame_errors_min_sum = torch.zeros(len(SNRs)).to(device)
nb_frames_min_sum = torch.zeros(len(SNRs)).to(device)
bit_errors_hard = torch.zeros(len(SNRs)).to(device)
nb_bits_hard = torch.zeros(len(SNRs)).to(device)
bit_errors_min_sum = torch.zeros(len(SNRs)).to(device)
nb_bits_min_sum = torch.zeros(len(SNRs)).to(device)
for i, snr in enumerate(SNRs):
            while bit_errors_min_sum[i] < req_err:
                # messages = torch.randint(0, 2, (test_size, k))
                codewords = torch.ones((test_size, n)).to(device)
                BPSK_codewords = -1*torch.ones_like(codewords).to(device)
                # codewords = messages @ G % 2
                # codewords = codewords.to(device)
                # BPSK_codewords = (0.5 - codewords) * 2
                sigma = torch.sqrt(torch.tensor(1) / (2 * (k/n) * 10**(snr/10)))
                noise = sigma * torch.randn(test_size, n).to(device)
                soft_input = BPSK_codewords + noise
                frame_errors_hard[i] += torch.sum(torch.any((-(torch.sign(soft_input[:,n-k:]) - 1)/2) != codewords[:,n-k:], axis=1))
                frame_errors_min_sum[i] += torch.sum(torch.any(torch.heaviside(ms(soft_input) - 0.5, torch.tensor([0.]).to(device)) != codewords, axis=1))
                bit_errors_hard[i] += torch.count_nonzero((-(torch.sign(soft_input) - 1)/2) != codewords)
                bit_errors_min_sum[i] += torch.count_nonzero(torch.heaviside(ms(soft_input) - 0.5, torch.tensor([0.]).to(device)) != codewords)
                nb_bits_hard[i] += test_size * n
                nb_bits_min_sum[i] += test_size * n
                nb_frames_hard[i] += test_size
                nb_frames_min_sum[i] += test_size
                print(f'SNR: {snr}, {100*bit_errors_min_sum[i]/req_err :.3f}%, test BER: {bit_errors_min_sum[i] / nb_bits_min_sum[i]}, test FER: {frame_errors_min_sum[i] / nb_frames_min_sum[i]}                ', end='\r')
            print('\n')

IndexError: index out of range in self

In [18]:
class weightConstraint(object):
    def __init__(self):
        pass
    
    def __call__(self,module):
        if hasattr(module,'weight'):
            print("Entered")
            W_cv=module.W_cv
            W_cv = torch.clamp(W_cv, min=0)
            module.W_cv=W_cv
            B_cv=module.B_cv
            B_cv = torch.clamp(W_cv, min=0)
            module.B_cv=B_cv
            W_vc=module.W_vc
            W_vc = torch.clamp(W_vc, min=0)
            module.W_vc=W_vc

In [19]:
constraints=weightConstraint()

In [ ]:
train_size = 50000
n_upd = 500
bit_errors_nn = torch.zeros((n_upd, len(SNRs))).to(device)
nb_bits_nn = torch.zeros((n_upd, len(SNRs))).to(device)
ber_list = []

for upd in range(n_upd):
    print(f'TRAINING ITERATION #{upd+1}/{n_upd}')
    training_loader, test_loader = update_loaders()
    train(training_loader, test_loader, decoder, criterion, opt, n_epochs=3)

    if (np.mod(upd+1,5)==0):
        decoder.eval()
        for i, snr in enumerate(SNRs):
            while bit_errors_nn[upd][i] < req_err:
                messages = torch.randint(0, 2, (test_size, k))
                codewords = messages @ G % 2
                codewords = codewords.to(device)
                BPSK_codewords = (0.5 - codewords) * 2
                # codewords = torch.ones((test_size, n)).to(device)
                # BPSK_codewords = -1*torch.ones_like(codewords).to(device)
                sigma = torch.sqrt(torch.tensor(1) / (2 * (np.float(k)/np.float(n)) * 10**(snr/10)))
                noise = sigma * torch.randn(test_size, n).to(device)
                soft_input = BPSK_codewords + noise
                bit_errors_nn[upd][i] += torch.count_nonzero(torch.heaviside(decoder(soft_input) - 0.5, torch.tensor([0.]).to(device)) != codewords)
                nb_bits_nn[upd][i] += test_size * n
                print(f'SNR: {snr}, {100*bit_errors_nn[upd][i]/req_err :.3f}%, test BER: {bit_errors_nn[upd][i] / nb_bits_nn[upd][i]}                ', end='\r')
            print('\n')
        ber_list.append((bit_errors_nn[upd,-1].cpu() / nb_bits_nn[upd,-1].cpu()).numpy())    
        np.savetxt('ber.txt', ber_list)

TRAINING ITERATION #1/500


 33%|████████████████████████████████████████                                                                                | 1/3 [00:01<00:02,  1.45s/it]

[Epoch 1] train loss: 40.777; train acc: 14.52; test loss: 40.774; test acc: 14.53


 67%|████████████████████████████████████████████████████████████████████████████████                                        | 2/3 [00:02<00:01,  1.44s/it]

[Epoch 2] train loss: 40.777; train acc: 14.52; test loss: 40.773; test acc: 14.53


...

 67%|████████████████████████████████████████████████████████████████████████████████                                        | 2/3 [00:02<00:01,  1.40s/it]

[Epoch 2] train loss: 40.776; train acc: 14.52; test loss: 40.772; test acc: 14.53


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.41s/it]


[Epoch 3] train loss: 40.776; train acc: 14.52; test loss: 40.772; test acc: 14.53
TRAINING ITERATION #5/500


  0%|                                                                                                                                | 0/3 [00:00<?, ?it/s]

In [ ]:
plt.figure(figsize=(16,10))
plt.semilogy(SNRs, bit_errors_min_sum.cpu() / nb_bits_min_sum.cpu(), label='Classical Min-sum')
plt.semilogy(SNRs, bit_errors_hard.cpu() / nb_bits_hard.cpu(), label='hard decision')
# for i in range(10):
plt.semilogy(SNRs, bit_errors_nn[77].cpu() / nb_bits_nn[77].cpu(), label=f'NN')
plt.legend()
plt.show()

In [ ]:
decoder(soft_input)[0] - 0.5

In [ ]:
codewords[0]

In [ ]:
decoder.W_cv

In [ ]:
decoder.B_cv